In [7]:
import numpy as np
import cv2 as cv 

def P4_rgb2gray(img) :
    b, g, r = cv.split(img)
    img = 0.229 * r + 0.578 * g + 0.144 * b
    img = cv.merge([b, g, r])
    return img

img = cv.imread('images/shenzhen.png', cv.IMREAD_COLOR)
img_gray = P4_rgb2gray(img).astype(np.uint8)

cv.imshow('img_gray', img_gray)
cv.waitKey(0)

-1

In [105]:
import cv2 as cv 
import numpy as np

img = cv.imread('images/blue_sign.png', cv.IMREAD_COLOR)
rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)

blue_low = np.array([0, 0, 150])
blue_up = np.array([150, 240, 255])

mask = cv.inRange(rgb, blue_low, blue_up)
img2 = cv.bitwise_and(img, img, mask = mask)

concatenate = np.concatenate((img, img2), axis = 1)
cv.imshow('concatenate', concatenate)
cv.waitKey(0)

-1

In [103]:
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
img = cv.imread('images/blue_sign.png')[ : , : , : : -1]
hsv = cv.cvtColor(img, cv.COLOR_RGB2HSV)

low_blue = np.array([100, 43, 46])
up_blue = np.array([124, 255, 240])

mask = cv.inRange(hsv, low_blue, up_blue)
img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
img2 = cv.bitwise_and(img, img, mask = mask)

concatenate = np.concatenate((img, img2), axis = 1)
cv.imshow('concatenate', concatenate)
cv.waitKey(0)

-1

In [102]:
import numpy as np
import cv2 as cv 
star_img = cv.imread('images/star.png', cv.IMREAD_GRAYSCALE)
moments = cv.moments(star_img)
HuMoments = cv.HuMoments(moments)

#平移
M = np.float32([[1, 0, 50], [0, 1, 30]])
rows, cols = star_img.shape[0], star_img.shape[1]
star_img1 = cv.warpAffine(star_img, M, (cols, rows))
moments = cv.moments(star_img1)
HuMoments1 = cv.HuMoments(moments)
delta1 =  HuMoments1 - HuMoments
print(delta1)

#旋转
star_img2 = np.rot90(star_img)
moments = cv.moments(star_img2)
HuMoments2 = cv.HuMoments(moments)
delta2 = HuMoments2 - HuMoments
print(delta2)

#尺度
star_img3 = cv.resize(star_img, (300, 300))
moments = cv.moments(star_img3)
HuMoments3 = cv.HuMoments(moments)
delta3 = HuMoments3 - HuMoments
print(delta3)


[[ 1.08420217e-19]
 [ 8.89219158e-24]
 [ 2.06997102e-26]
 [-3.00398233e-27]
 [ 4.86082411e-41]
 [ 1.11703937e-32]
 [-2.67697332e-39]]
[[ 2.16840434e-19]
 [-1.28212995e-23]
 [-7.97948583e-26]
 [ 8.81630948e-27]
 [ 4.95947552e-40]
 [ 5.97808655e-31]
 [-3.12581483e-39]]
[[1.56914471e-06]
 [2.67185822e-09]
 [2.59890348e-15]
 [2.50988314e-15]
 [2.17424360e-28]
 [7.40875755e-19]
 [2.14289027e-29]]


In [108]:
import cv2 as cv 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
img_planes = cv.imread('images/planes.png', 0)

img = np.zeros((img_planes.shape[0], img_planes.shape[1], 3))
contours, hierarphy = cv.findContours(img_planes, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)#contours得到宽和高
cv.drawContours(img, contours,  -1, (0, 0, 255), 1)

contours_tmp = []
for k in range(len(contours)) :
    X = contours[k].squeeze()
    min_x, min_y, max_x, max_y = np.min(X, 0)[0], np.min(X, 0)[1], np.max(X, 0)[0], np.max(X, 0)[1]
    contours_tmp.append([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])
#cv2.line画矩形
# for i in range(len(contours_tmp)) :
#     cv.line(img, (contours_tmp[i][3][0], contours_tmp[i][3][1]), (contours_tmp[i][0][0], contours_tmp[i][0][1]), (0, 0, 255), 1)
#     for j in range(len(contours_tmp[i]) - 1) :
#         cv.line(img, (contours_tmp[i][j][0], contours_tmp[i][j][1]), (contours_tmp[i][j + 1][0], contours_tmp[i][j + 1][1]), (0, 0, 255), 1)
#         cv.imshow('img', img)
#         cv.waitKey(1)
# cv.waitKey(0)
for i in range(len(contours_tmp)) :
    cv.rectangle(img, (contours_tmp[i][0][0], contours_tmp[i][0][1]), (contours_tmp[i][2][0], contours_tmp[i][2][1]), (0, 0, 255))

img_plane = cv.imread('images/one_plane.png', 0)
sub_images = []
deltas = []
#单架飞机的Hu矩
moments_plane = cv.moments(img_plane)
Hu_plane = cv.HuMoments(moments_plane)
Hu_planes = []

for k in range(len(contours_tmp)) :
    sub_image = img_planes[contours_tmp[k][0][1] : contours_tmp[k][2][1], contours_tmp[k][0][0] : contours_tmp[k][2][0]].astype(np.float32)
    sub_images.append(sub_image)
    
    moments = cv.moments(sub_image)
    Humoments = cv.HuMoments(moments)
    Hu_planes.append(Humoments)
    #定义Hu矩的距离为欧几里得距离
    delta = np.square(Humoments - Hu_plane)
    deltas.append(np.sum(delta))
    
    plt.subplot(1, 4, k + 1)
    plt.title(f'sub {k + 1}')
    plt.axis('off')
    plt.tight_layout()
    plt.imshow(sub_image, 'gray')

plt.figure()
the_nearest = deltas.index(min(deltas))
plt.subplot(1, 2, 1), plt.imshow(img_plane, 'gray'), plt.title('Original'),plt.axis('off')
plt.subplot(1, 2, 2), plt.imshow(sub_images[the_nearest], 'gray'), plt.title('Matched'),plt.axis('off')

cv.imshow('contours', img)
cv.waitKey(0)

Using matplotlib backend: Qt5Agg


-1

In [33]:
import cv2 as cv 

def P4_contour_code(pointList) :
    Normalized_chain_code = []
    Differential_code = []
    direction = [[1, 0], [1, 1], [0, 1], [-1, 1], [-1, 0], [-1, -1], [0, -1], [1, -1]]

    for k in range(1, len(contours_tmp)) :
        for t in range(len(direction)) :
            if (contours_tmp[k][0] == (contours_tmp[k - 1][0] + direction[t][0])) and (contours_tmp[k][1] == (contours_tmp[k - 1][1] + direction[t][1])) :
                Normalized_chain_code.append(t)
    Normalized_chain_code.append(Normalized_chain_code[0])

    for i in range(1, len(Normalized_chain_code)) :
        if Normalized_chain_code[i] < Normalized_chain_code[i - 1] :
            Differential_code.append(Normalized_chain_code[i] + 8 - Normalized_chain_code[i - 1])
        else :
            Differential_code.append(Normalized_chain_code[i] - Normalized_chain_code[i - 1])
    return Differential_code, Normalized_chain_code

star = cv.imread('images/star.png', cv.IMREAD_GRAYSCALE)
contours, hierarphy = cv.findContours(star, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
pointList = contours[0].squeeze()

Chain_code, Differential_code = P4_contour_code(pointList)
print('归一化链码: \n',"".join(map(str, Chain_code)))
print('归一化差分码: \n',"".join(map(str, Differential_code)))

归一化链码: 
 0000727612077117000072701601171207702701500000000000000000000007277270071170170110612167106711707101770216711627072772710207611170527726270000000711700110000000000000000010167117061216661277111000000000000000000000000000010107275277062111066017011000000000000000000000000000000000000011711606100072157000071011000000710670216007717717216012700007267612070071061200772711072707717170711100072771672700072107270071150007101626270000100100000777262707112772166012105105116116101101772707061017072107270007210007270716702167261001611761117000177277211707021606017711617270151215020512167172167116117612161100711775121600007277270007270000161001172772772500007276121600007277611512160612061017000511700727110001602767117000011720627726270077121766270000011710007106702717017721012704012177070611772621702716117702070711501611626217116277250007270721000727000071176121607211601107707727000711707270011000711616072707200270727705127170711172070000067260207600111270511610170110727007167100720017117